# Sistema de predicción de ventas (ARIMA)

Este notebook resuelve el proyecto de **Series de tiempo** (ventas) siguiendo los pasos del enunciado.

**Dataset (URL):**  
https://breathecode.herokuapp.com/asset/internal-link?id=2546&path=sales.csv

## Objetivos
1. Cargar el dataset y construir la serie temporal (índice de fechas + variable de ventas).
2. Analizar: **tensor (frecuencia)**, **tendencia**, **estacionariedad** y **ruido/variabilidad**.
3. Entrenar un **ARIMA** (idealmente con `auto_arima`).
4. Predecir en el conjunto de test y medir el rendimiento.
5. Guardar el modelo entrenado.

In [ ]:
# ==============================
# 0) Imports y configuración
# ==============================

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error, mean_squared_error

# Tests y utilidades de series temporales
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose

# Intentamos usar auto_arima (recomendado). Si no está disponible, usamos statsmodels ARIMA + mini-grid.
AUTO_ARIMA_AVAILABLE = True
try:
    from pmdarima import auto_arima
except Exception as e:
    AUTO_ARIMA_AVAILABLE = False
    print("⚠️ No se pudo importar pmdarima/auto_arima. Se usará un fallback con statsmodels.")
    print("Detalle:", e)

print("auto_arima disponible:", AUTO_ARIMA_AVAILABLE)

In [ ]:
# ==============================
# 1) Carga del dataset
# ==============================

DATA_URL = "https://breathecode.herokuapp.com/asset/internal-link?id=2546&path=sales.csv"

df = pd.read_csv(DATA_URL)
print("Shape:", df.shape)
display(df.head())
print("\nColumnas:", list(df.columns))

## 2) Construcción de la serie temporal

La idea es:
- identificar la columna de fecha (si existe),
- identificar la columna de ventas,
- convertir la fecha a `datetime`,
- ordenar por fecha y establecerla como índice,
- y finalmente trabajar con una Serie `ts`.

> Si el dataset usa nombres de columnas diferentes a los esperados, el notebook intenta detectarlas.  
> Si no lo logra, te deja un bloque donde puedes asignarlas manualmente.

In [ ]:
# ==============================
# 2) Construcción de la serie temporal (detección automática)
# ==============================

def guess_date_col(columns):
    candidates = ["date", "ds", "datetime", "timestamp", "time", "month", "year_month", "period"]
    cols_lower = {c.lower(): c for c in columns}
    for cand in candidates:
        if cand in cols_lower:
            return cols_lower[cand]
    # Si no hay coincidencia directa, intentamos por dtype luego
    return None

def guess_target_col(columns):
    candidates = ["sales", "y", "value", "target", "qty", "quantity", "revenue", "ventas"]
    cols_lower = {c.lower(): c for c in columns}
    for cand in candidates:
        if cand in cols_lower:
            return cols_lower[cand]
    return None

date_col = guess_date_col(df.columns)
target_col = guess_target_col(df.columns)

print("Date col (guess):", date_col)
print("Target col (guess):", target_col)

# Heurística adicional: si no detectó fecha, probamos columnas tipo object que parezcan fecha.
if date_col is None:
    for c in df.columns:
        if df[c].dtype == "object":
            sample = df[c].dropna().astype(str).head(20).tolist()
            # Probamos convertir un pequeño sample a datetime
            try:
                pd.to_datetime(sample, errors="raise")
                date_col = c
                break
            except Exception:
                continue

print("Date col (after heuristic):", date_col)

# Si no detecta target, tomamos la última columna numérica como fallback (no ideal, pero útil)
if target_col is None:
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    if len(num_cols) > 0:
        target_col = num_cols[-1]

print("Target col (after heuristic):", target_col)

# Si algo falla: asignación manual (edita aquí)
if date_col is None or target_col is None:
    raise ValueError(
        "No pude detectar automáticamente la columna de fecha o de ventas. "
        "Revisa df.columns y asigna date_col y target_col manualmente."
    )

# Parseo de fecha
df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
df = df.dropna(subset=[date_col, target_col]).copy()

# Orden e índice
df = df.sort_values(date_col)
df = df.set_index(date_col)

# Serie temporal final
ts = df[target_col].astype(float)

print("Rango de fechas:", ts.index.min(), "→", ts.index.max())
print("Nulos en ts:", ts.isna().sum())
display(ts.head())

In [ ]:
# ==============================
# 3) Tensor (frecuencia) y visualización
# ==============================

# Intentamos inferir la frecuencia de la serie
freq = pd.infer_freq(ts.index)
print("Frecuencia inferida (tensor):", freq)

# Si no se puede inferir, calculamos el delta más común
if freq is None:
    deltas = ts.index.to_series().diff().dropna()
    most_common = deltas.value_counts().head(1)
    print("Delta más común:", most_common.index[0], "→", most_common.iloc[0], "veces")

# Plot básico
plt.figure(figsize=(12, 4))
plt.plot(ts.index, ts.values)
plt.title("Serie temporal de ventas")
plt.xlabel("Fecha")
plt.ylabel(target_col)
plt.tight_layout()
plt.show()

## 4) Análisis: tendencia, estacionariedad y ruido/variabilidad

- **Tendencia:** la estimamos con una media móvil y también con la pendiente (regresión lineal simple).
- **¿Es estacionaria?:** aplicamos **Dickey-Fuller (ADF)**. Si `p-value < 0.05`, consideramos estacionaria.
- **Ruido/variabilidad:** miramos el componente residual de una descomposición estacional (si tiene sentido con la frecuencia).

In [ ]:
# ==============================
# 4) Tendencia (rolling + pendiente) y Estacionariedad (ADF)
# ==============================

# Media móvil para ver tendencia (ventana adaptativa)
n = len(ts)
window = max(3, int(round(n * 0.05)))  # ~5% del tamaño, mínimo 3
rolling_mean = ts.rolling(window=window).mean()

plt.figure(figsize=(12, 4))
plt.plot(ts.index, ts.values, label="Ventas")
plt.plot(rolling_mean.index, rolling_mean.values, label=f"Media móvil (window={window})")
plt.title("Tendencia (aprox) con media móvil")
plt.xlabel("Fecha")
plt.ylabel(target_col)
plt.legend()
plt.tight_layout()
plt.show()

# Pendiente (regresión lineal simple sobre el tiempo)
x = np.arange(len(ts))
slope = np.polyfit(x, ts.values, 1)[0]
print(f"Pendiente aproximada (slope): {slope:.6f} ->",
      "tendencia creciente" if slope > 0 else "tendencia decreciente" if slope < 0 else "sin tendencia clara")

# Dickey-Fuller
def adf_test(series):
    res = adfuller(series.dropna(), autolag="AIC")
    return {
        "test_statistic": res[0],
        "p_value": res[1],
        "lags": res[2],
        "nobs": res[3],
        "critical_values": res[4]
    }

adf = adf_test(ts)
print("\nADF Test:")
print("  Test statistic:", adf["test_statistic"])
print("  p-value       :", adf["p_value"])
print("  # lags        :", adf["lags"])
print("  # obs         :", adf["nobs"])
print("  Critical values:", adf["critical_values"])

print("\nConclusión (estacionariedad):",
      "✅ Estacionaria (p<0.05)" if adf["p_value"] < 0.05 else "❌ No estacionaria (p>=0.05)")

In [ ]:
# ==============================
# 5) Descomposición estacional (si aplica)
# ==============================

# Elegimos un 'period' razonable según la frecuencia inferida.
# Ajusta si tu serie tiene otra estacionalidad.
period = None
if freq is not None:
    # Mensual -> 12; Semanal -> 52; Diario -> 7 (semana); Horario -> 24
    if "M" in freq:
        period = 12
    elif "W" in freq:
        period = 52
    elif "D" in freq:
        period = 7
    elif "H" in freq:
        period = 24

print("Period sugerido para descomposición:", period)

if period is None or len(ts) < 2 * (period if period else 1):
    print("⚠️ No se hace descomposición (frecuencia no inferida o serie muy corta para un period estable).")
else:
    decomposition = seasonal_decompose(ts, period=period, model="additive")
    trend = decomposition.trend
    seasonal = decomposition.seasonal
    resid = decomposition.resid

    # Plot de componentes
    fig = decomposition.plot()
    fig.set_size_inches(12, 8)
    plt.tight_layout()
    plt.show()

    # Ruido/variabilidad: resumen del residuo
    resid_clean = resid.dropna()
    print("Residuo: media =", resid_clean.mean(), "| std =", resid_clean.std())
    print("Interpretación rápida: a mayor std del residuo, mayor variabilidad/noise no explicado por tendencia/estacionalidad.")

## 6) Train/Test Split (sin mezclar)

En series de tiempo **no** se debe mezclar aleatoriamente: se usa un split temporal.  
Tomaremos el **último 20%** como test (puedes cambiar `test_size`).

In [ ]:
# ==============================
# 6) Split temporal: 80/20
# ==============================

test_size = 0.2
split_idx = int(len(ts) * (1 - test_size))

train_ts = ts.iloc[:split_idx]
test_ts  = ts.iloc[split_idx:]

print("Train:", train_ts.index.min(), "→", train_ts.index.max(), "| n =", len(train_ts))
print("Test :", test_ts.index.min(),  "→", test_ts.index.max(),  "| n =", len(test_ts))

plt.figure(figsize=(12, 4))
plt.plot(train_ts.index, train_ts.values, label="Train")
plt.plot(test_ts.index, test_ts.values, label="Test")
plt.title("Split temporal (train/test)")
plt.xlabel("Fecha")
plt.ylabel(target_col)
plt.legend()
plt.tight_layout()
plt.show()

## 7) Entrenamiento ARIMA (búsqueda de parámetros)

- Si `auto_arima` está disponible, lo usamos para estimar automáticamente `(p, d, q)` y (si aplica) estacionalidad.
- Si no, hacemos un mini grid-search con `statsmodels` (más lento y básico).

In [ ]:
# ==============================
# 7) Entrenamiento ARIMA
# ==============================

model = None

if AUTO_ARIMA_AVAILABLE:
    # Reglas simples para estacionalidad: si la frecuencia es mensual, usamos m=12; semanal m=52; diaria m=7.
    m = 1
    seasonal = False
    if freq is not None:
        if "M" in freq:
            m = 12; seasonal = True
        elif "W" in freq:
            m = 52; seasonal = True
        elif "D" in freq:
            m = 7; seasonal = True

    print(f"Configuración auto_arima: seasonal={seasonal}, m={m}")
    model = auto_arima(
        train_ts,
        seasonal=seasonal,
        m=m,
        trace=True,
        error_action="ignore",
        suppress_warnings=True,
        stepwise=True
    )
    print("\nMejor modelo:", model.order, "estacional:", getattr(model, "seasonal_order", None))

else:
    # Fallback simple con statsmodels
    from statsmodels.tsa.arima.model import ARIMA

    best_aic = np.inf
    best_order = None
    best_model = None

    # Grid pequeño para no tardar demasiado
    for p in range(0, 4):
        for d in range(0, 3):
            for q in range(0, 4):
                try:
                    tmp = ARIMA(train_ts, order=(p, d, q)).fit()
                    if tmp.aic < best_aic:
                        best_aic = tmp.aic
                        best_order = (p, d, q)
                        best_model = tmp
                except Exception:
                    continue

    model = best_model
    print("Mejor order:", best_order, "| AIC:", best_aic)

model

## 8) Predicción en el conjunto de test y evaluación

Calculamos métricas típicas:
- **MAE**
- **RMSE**
- (Opcional) **MAPE** si no hay ceros.

In [ ]:
# ==============================
# 8) Forecast sobre el horizonte del test
# ==============================

h = len(test_ts)

if AUTO_ARIMA_AVAILABLE:
    y_pred = pd.Series(model.predict(n_periods=h), index=test_ts.index)
else:
    # statsmodels ARIMA
    fc = model.forecast(steps=h)
    y_pred = pd.Series(fc, index=test_ts.index)

# Métricas
mae = mean_absolute_error(test_ts, y_pred)
rmse = mean_squared_error(test_ts, y_pred, squared=False)

# MAPE (cuidado con ceros)
eps = 1e-9
mape = np.mean(np.abs((test_ts - y_pred) / (np.abs(test_ts) + eps))) * 100

print(f"MAE : {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAPE: {mape:.2f}%")

# Plot comparativo
plt.figure(figsize=(12, 4))
plt.plot(train_ts.index, train_ts.values, label="Train")
plt.plot(test_ts.index, test_ts.values, label="Test (real)")
plt.plot(y_pred.index, y_pred.values, label="Forecast (pred)")
plt.title("ARIMA: real vs predicción (test)")
plt.xlabel("Fecha")
plt.ylabel(target_col)
plt.legend()
plt.tight_layout()
plt.show()

## 9) Guardado del modelo

Se guardará en `models/` dentro del repo.
- Con `auto_arima` (pmdarima) es común usar `model.save(...)`.
- Con el fallback de statsmodels, usamos `pickle`.

In [ ]:
# ==============================
# 9) Guardado del modelo
# ==============================

from pathlib import Path
import pickle

models_dir = Path("models")
models_dir.mkdir(exist_ok=True)

if AUTO_ARIMA_AVAILABLE:
    out_path = models_dir / "arima_sales_model.pkl"
    model.save(str(out_path))
    print("✅ Modelo guardado en:", out_path)
else:
    out_path = models_dir / "arima_sales_model.pkl"
    with open(out_path, "wb") as f:
        pickle.dump(model, f)
    print("✅ Modelo guardado en:", out_path)

## Checklist contra el enunciado

- ✅ Carga del dataset y creación de serie temporal (`ts`).
- ✅ Tensor (frecuencia) con `infer_freq` y/o delta más común.
- ✅ Tendencia (media móvil + pendiente).
- ✅ Estacionariedad (ADF).
- ✅ Variabilidad / ruido (residuo de descomposición, si aplica).
- ✅ ARIMA entrenado con train (auto_arima o fallback).
- ✅ Predicción en test + métricas + gráfico.
- ✅ Modelo guardado en `models/`.

Si quieres, en el siguiente paso podemos:
- ajustar estacionalidad (`m`) si tu frecuencia real no es mensual,
- probar transformaciones (log/Box-Cox),
- o comparar contra un baseline (naive, moving average).